```
This software is part of GPU Ocean. 

Copyright (C) 2017, 2018 SINTEF Digital
Copyright (C) 2017, 2018 Norwegian Meteorological Institute

This notebook implements the Rossby adjustment for variable Rossby radius 
of deformation test case, as reported under Case C in Test Cases for 
Rotational Shallow-Water Schemes by Holm, Brodtkorb, Broström, Christensen 
and Sætra.

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <http://www.gnu.org/licenses/>.
```

# Case C: The Adjustment Problem for Variable Rossby Radius of Deformation

In this notebook, we investigate different properties with our numerical schemes related to geostrophic balance when we vary the water depth. By varying the water depth, we also vary the Rossby radius. Geostrophic balance represent steady state solutions where the pressure gradients are balanced by the Coriolis forces.

This notebook runs the simulations and write the results to NetCDF files. A json-file is also created, containing an overview of the netcdf-files produced in this experiment. This is useful for the post-processing and plotting, which is done by the notebook `CaseC_VariableRossbyRadius_Plotting.ipynb`.



# Setting up the environment

In [ ]:
#Lets have matplotlib "inline"
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

#Import packages we need
import numpy as np
from matplotlib import animation, rc
from matplotlib import pyplot as plt
from matplotlib import gridspec as gridspec

import os, pyopencl, datetime, sys
import json
import subprocess

# requires netcdf4-python (netcdf4-python.googlecode.com)
from netCDF4 import Dataset as NetCDFFile

#Set large figure sizes
rc('figure', figsize=(16.0, 12.0))
rc('animation', html='html5')

#Finally, import our simulator
from SWESimulators import FBL, CTCS, KP07, CDKLM16, SimWriter, PlotHelper, Common
from SWESimulators.BathymetryAndICs import *

In [ ]:
#Make sure we get compiler output from OpenCL
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"

#Set which CL device to use, and disable kernel caching
if (str.lower(sys.platform).startswith("linux")):
    os.environ["PYOPENCL_CTX"] = "0"
else:
    os.environ["PYOPENCL_CTX"] = "1"
os.environ["CUDA_CACHE_DISABLE"] = "1"
os.environ["PYOPENCL_COMPILER_OUTPUT"] = "1"
os.environ["PYOPENCL_NO_CACHE"] = "1"

#Create OpenCL context
cl_ctx = pyopencl.create_some_context()
print "Using ", cl_ctx.devices[0].name

In [ ]:
#Create output directory for images
imgdir='images_' + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")
os.makedirs(imgdir)
print "Saving images to " + imgdir

# Setting common parameters



In [ ]:
# Common parameters 

nx = 800
ny = 1000

dx = 50000
dy = 50000

dt = 100
g = 9.81
f = 1.2e-4
r = 0.0
multi_H0 = np.array(range(100, 5200, 500))*1.0
print(multi_H0)
A = 0.0 # A is diffusion coefficient multiplied by depth.

wind = Common.WindStressParams(type=99)

## Setting up boundary conditions
sponge = [10, 10, 10, 10]
boundaryConditions = Common.BoundaryConditions(3,3,3,3, spongeCells=sponge)

ghosts = [10, 10, 10, 10]
dataShape = (ny + ghosts[0]+ghosts[2], 
             nx + ghosts[1]+ghosts[3])
validDomain =  [10, 10, 10, 10]

geoBalancePlot = True

T = 40
sub_T = 2*600*dt
make_netCDF = True

# For plotting:
#Calculate radius from center of bump for plotting
x_center = dx*nx/2.0
y_center = dy*ny/2.0
y_coords, x_coords = np.mgrid[0:ny*dy:dy, 0:nx*dx:dx]
x_coords = np.subtract(x_coords, x_center)
y_coords = np.subtract(y_coords, y_center)
radius = np.sqrt(np.multiply(x_coords, x_coords) + np.multiply(y_coords, y_coords))

In [ ]:
def setUpInitialStructure(file_name):
    git_hash = str.strip(subprocess.check_output(['git', 'rev-parse', 'HEAD']))

    main_data = {'git_hash': git_hash, \
                 'FBL':   [],  \
                 'CTCS':  [],  \
                 'KP07':  [],  \
                 'CDKLM': [],  \
                 'timestamp': datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S"), \
                 'GPU': cl_ctx.devices[0].name}
    
    with open(file_name, 'w') as fout:
        json.dump(main_data, fout)
        
def addResults(file_name, sim_name, netcdf_name, depth, eta_max, eta_init_max, T_end, dt):
    
    sim_data = {'sim_name': sim_name, 'netcdf_file_name': netcdf_name, \
                'depth': float(depth), 'eta_max': float(eta_max), 'eta_init_max': float(eta_init_max), \
                'T_end': float(T_end), 'dt': float(dt)}
    with open(file_name, mode='r+') as json_file:
        json_element =  json.load(json_file)
        json_file.seek(0)
        #print json_element
        json_element[sim_name].append(sim_data)
        #print json_element
        json.dump(json_element, json_file)
    
    print "Wrote " + sim_name + " with H0="+str(depth) + " to json"
    print "Relative eta_max/eta_init_max: ", eta_max/eta_init_max
    print "(dt, T_end): ", (dt, T_end)
    print "------------------------------"


jsondir = "rossbyAdjustmentResults/"
json_file_name = jsondir + "all_simulators_" + datetime.datetime.now().strftime("%Y_%m_%d-%H_%M_%S")  +".json"
setUpInitialStructure(json_file_name)
print json_file_name
#json_file_name = jsondir + "all_simulators_2018_01_25-13_10_27.json"
#addResults(json_file_name, 'CTCS', 'netcfd_NaMe', 10, 1, 2, 100000, 0.001)


##### Creating initial conditions

Staggered and unstaggered grids are placed so that cell centers are on the same place. Keep therefore in mind that the velocities are defined at different positions for the two different grid types!

In [ ]:
def initialConditions(eta0, nx, ny, dx, dy, halo_x, halo_y):
    print "Making initial conditions"
    x_center = dx*nx/2.0
    y_center = dy*ny/2.0
    y_center1 = dy*(ny-100)/2.0
    y_center2 = dy*(ny+100)/2.0

    for j in range(-halo_y, ny+halo_y):
        for i in range(-halo_x, nx+halo_x):
            x = dx*i - x_center 
            y = dy*j - y_center 
            y1 = dy*j - y_center1
            y2 = dy*j - y_center2

            # Initial smooth step
            inirad = np.sqrt(x**2 + y**2)
            inirad1 = np.sqrt(x**2 + y1**2)
            inirad2 = np.sqrt(x**2 + y2**2)
            L = 15*dx
            D = 50*dx
            etaamp = 0.2
            
            # Add the the initial condition to the provided array
            #eta0[j+1, i+1] += 0.5*etaamp*(1.0+np.tanh((-inirad+D)/L))
            eta0[j+halo_y, i+halo_x] += 0.5*etaamp*(1.0+np.tanh((-inirad+D)/L))
            

### Evaluating the "steady state"ness of the solution by looking at Klein-Gordon

In [ ]:
"""
Returns d/dx(0.5*g*h*h) + fhv
"""
def geostrophicBalanceEta(eta, H, hu, hv, nx, ny, dx, dy, f, g):
    return geostrophicBalance(eta+H, hu, hv, nx, ny, dx, dy, f, g)

def geostrophicBalance(h, hu, hv, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells in input arrays
    A = hu*hv/h
    B = hu*hu/h
    C = 0.5*g*h*h
    D = -f*hv
    
    Ay = np.zeros_like(A)
    Bx = np.zeros_like(B)
    Cx = np.zeros_like(C)
    
    Ay[1:-1,:] = (A[:-2,:] - A[2:,:])/(2*dy)
    Bx[:, 1:-1] = (B[:,:-2] - B[:,2:])/(2*dx)
    Cx[:, 1:-1] = (C[:,:-2] - C[:,2:])/(2*dx)
    
    geoBalance = Cx - D
    return geoBalance

def geostrophicBalanceStaggered(eta, H, hu_s, hv_s, nx, ny, dx, dy, f, g):
    # Expect 0 ghost cells only
       
    h = eta + H
    hu = 0.5*(hu_s[:, :-1] + hu_s[:, 1:])
    hv = 0.5*(hv_s[:-1, :] + hv_s[1:, :])
    return geostrophicBalance(h, hu, hv, nx, ny, dx, dy, f, g )

## Here, we assume that ghost cells are a part of the picture, and that there are 10 in all direction
def evaluateBalance(eta_tot, hu_tot, hv_tot, H0_tot):
    staggered = not (eta_tot.shape == hu_tot.shape)
    eta = eta_tot[10:-10, 10:-10]
    H   =  H0_tot[10:-10, 10:-10]
    hu  =  hu_tot[10:-10, 10:-10]
    hv  =  hv_tot[10:-10, 10:-10]
    if staggered:
        geoBalance_x = geostrophicBalanceStaggered(eta,   H,   hu,   hv,   nx, ny, dx, dy, f, g)
        geoBalance_y = geostrophicBalanceStaggered(eta.T, H.T, hv.T, hu.T, nx, ny, dx, dy, f, g)
    else:
        geoBalance_x = geostrophicBalanceEta(eta,   H,   hu,   hv,   nx, ny, dx, dy, f, g)
        geoBalance_y = geostrophicBalanceEta(eta.T, H.T, hv.T, hu.T, nx, ny, dx, dy, f, g)
    print "max geobalances - (x,y): ", (np.max(geoBalance_x), np.max(geoBalance_y))

    if geoBalancePlot:
        fig = plt.figure(figsize=(10, 4))
        plt.subplot(1,2,1)
        plt.imshow(geoBalance_x, interpolation="none", origin='lower')
        plt.title("x-direction")
        plt.colorbar()
        plt.subplot(1,2,2)
        plt.imshow(geoBalance_y.T, interpolation="none", origin='lower')
        plt.title("y-direction")
        plt.colorbar()

# Run simulations

## Looping over different depth values

In [ ]:
# Forward-Backward-Linear 

reload(FBL)

fbl_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
initialConditions(fbl_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

fbl_u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
fbl_v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');

fbl_dt = dt
fbl_sub_T = sub_T

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

for i in range(len(multi_H0)):
    H0 = multi_H0[i]

    fbl_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
    if H0 == 600:
        fbl_sub_T = fbl_sub_T/2
    if H0 == 1600:
        fbl_sub_T = fbl_sub_T/2
    if H0 == 3600:
        fbl_dt = fbl_dt/2
        fbl_sub_T = fbl_sub_T/2
    if H0 == 4600:
        fbl_dt = fbl_dt/2
        
    #Initialize simulator
    fbl_sim = FBL.FBL(cl_ctx, \
                      fbl_h0, fbl_eta0, fbl_u0, fbl_v0, \
                      nx, ny, \
                      dx, dy, fbl_dt, \
                      g, f, r, \
                      wind_stress=wind, \
                      boundary_conditions=boundaryConditions, \
                      write_netcdf=make_netCDF
                     )

    print "Starting FBL with H0 = " + str(H0)
    
    t = fbl_sim.step(T*fbl_sub_T)

    # Computing the interesting values:    
    fbl_eta1, u1, v1 = fbl_sim.download()
    #evaluateBalance(fbl_eta1, u1, v1, fbl_h0)
    fbl_sim.cleanUp()
    
    eta_max = np.max(fbl_eta1)
    eta_init_max = np.max(fbl_eta0)
        
    addResults(json_file_name, "FBL", fbl_sim.sim_writer.output_file_name, H0, eta_max, eta_init_max, t, fbl_dt) 
    
    fbl_h0 = None
    fbl_eta1, u1, v1 = None, None, None
    #if 'fbl_sim' in globals():
    #    fbl_sim.cleanUp()

In [ ]:
#Centered-in-Time, Centered-in-space

reload(CTCS)

ctcs_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
initialConditions(ctcs_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

ctcs_u0 = np.zeros((dataShape[0], dataShape[1]+1), dtype=np.float32, order='C');
ctcs_v0 = np.zeros((dataShape[0]+1, dataShape[1]), dtype=np.float32, order='C');

ctcs_dt = dt
ctcs_sub_T = sub_T

# Bathymetry:
Bi = np.zeros((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C')

for i in range(len(multi_H0)):
    H0 = multi_H0[i]

    ctcs_h0 = np.ones(dataShape, dtype=np.float32, order='C') * H0;
    if H0 == 600:
        ctcs_sub_T = ctcs_sub_T/2
    if H0 == 1600:
        ctcs_sub_T = ctcs_sub_T/2
    if H0 == 3600:
        ctcs_dt = ctcs_dt/2
        ctcs_sub_T = ctcs_sub_T/2
    if H0 == 4600:
        ctcs_dt = ctcs_dt/2
        
    #Initialize simulator
    ctcs_sim = CTCS.CTCS(cl_ctx, \
                         ctcs_h0, ctcs_eta0, ctcs_u0, ctcs_v0, \
                         nx, ny, \
                         dx, dy, ctcs_dt, \
                         g, f, r, A, \
                         wind_stress=wind, \
                         boundary_conditions=boundaryConditions, \
                         write_netcdf=make_netCDF
                        )

    print "Starting CTCS with H0 = " + str(H0)
    
    t = ctcs_sim.step(T*ctcs_sub_T)

    # Computing the interesting values:    
    ctcs_eta1, u1, v1 = ctcs_sim.download()
    #evaluateBalance(ctcs_eta1, u1, v1, ctcs_h0)
    ctcs_sim.cleanUp()
    
    eta_max = np.max(ctcs_eta1)
    eta_init_max = np.max(ctcs_eta0)
        
    addResults(json_file_name, "CTCS", ctcs_sim.sim_writer.output_file_name, H0, eta_max, eta_init_max, t, ctcs_dt) 
    
    ctcs_h0 = None
    ctcs_eta1, u1, v1 = None, None, None
    #if 'ctcs_sim' in globals():
    #    ctcs_sim.cleanUp()

In [ ]:
# Kurganov-Petrova 2007

reload(KP07)

kp07_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
initialConditions(kp07_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

kp07_u0 = np.zeros((dataShape[0], dataShape[1]), dtype=np.float32, order='C');
kp07_v0 = np.zeros((dataShape[0], dataShape[1]), dtype=np.float32, order='C');

kp07_dt = dt
kp07_sub_T = sub_T


for i in range(len(multi_H0)):
    H0 = multi_H0[i]

    kp07_h0 = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C') * H0;
    if H0 == 600:
        kp07_sub_T = kp07_sub_T/2
    if H0 == 1600:
        kp07_sub_T = kp07_sub_T/2
    if H0 == 3600:
        kp07_dt = kp07_dt/2
        kp07_sub_T = kp07_sub_T/2
    if H0 == 4600:
        kp07_dt = kp07_dt/2
        
    #Initialize simulator
    kp07_sim = KP07.KP07(cl_ctx, \
                         kp07_eta0, kp07_h0, kp07_u0, kp07_v0, \
                         nx, ny, \
                         dx, dy, kp07_dt, \
                         g, f, r, \
                         wind_stress=wind, \
                         boundary_conditions=boundaryConditions, \
                         write_netcdf=make_netCDF
                        )

    print "Starting KP07 with H0 = " + str(H0)
    
    t = kp07_sim.step(T*kp07_sub_T)

    # Computing the interesting values:    
    kp07_eta1, u1, v1 = kp07_sim.download()
    #evaluateBalance(kp07_eta1, u1, v1, kp07_h0)
    kp07_sim.cleanUp()
    
    eta_max = np.max(kp07_eta1)
    eta_init_max = np.max(kp07_eta0)
        
    addResults(json_file_name, "KP07", kp07_sim.sim_writer.output_file_name, H0, eta_max, eta_init_max, t, kp07_dt) 
    
    kp07_h0 = None
    kp07_eta1, u1, v1 = None, None, None
    #if 'kp07_sim' in globals():
    #    kp07_sim.cleanUp()

In [ ]:
# CDKLM16

reload(CDKLM16)

cdklm_eta0 = np.zeros(dataShape, dtype=np.float32, order='C');
initialConditions(cdklm_eta0, nx, ny, dx, dy, ghosts[1], ghosts[0])

cdklm_u0 = np.zeros((dataShape[0], dataShape[1]), dtype=np.float32, order='C');
cdklm_v0 = np.zeros((dataShape[0], dataShape[1]), dtype=np.float32, order='C');

cdklm_dt = dt
cdklm_sub_T = sub_T

for i in range(len(multi_H0)):
    H0 = multi_H0[i]

    cdklm_h0 = np.ones((dataShape[0]+1, dataShape[1]+1), dtype=np.float32, order='C') * H0;
    if H0 == 600:
        cdklm_sub_T = cdklm_sub_T/2
    if H0 == 1600:
        cdklm_sub_T = cdklm_sub_T/2
    if H0 == 3600:
        cdklm_dt = cdklm_dt/2
        cdklm_sub_T = cdklm_sub_T/2
    if H0 == 4600:
        cdklm_dt = cdklm_dt/2
        
    #Initialize simulator
    cdklm_sim = CDKLM16.CDKLM16(cl_ctx, \
                         cdklm_eta0, cdklm_u0, cdklm_v0, cdklm_h0, \
                         nx, ny, \
                         dx, dy, cdklm_dt, \
                         g, f, r, \
                         wind_stress=wind, \
                         boundary_conditions=boundaryConditions, \
                         write_netcdf=make_netCDF
                        )

    print "Starting CDKLM16 with H0 = " + str(H0)
    
    t = cdklm_sim.step(T*cdklm_sub_T)

    # Computing the interesting values:    
    cdklm_eta1, u1, v1 = cdklm_sim.download()
    #evaluateBalance(cdklm_eta1, u1, v1, cdklm_h0)
    cdklm_sim.cleanUp()
    
    eta_max = np.max(cdklm_eta1)
    eta_init_max = np.max(cdklm_eta0)
        
    addResults(json_file_name, "CDKLM", cdklm_sim.sim_writer.output_file_name, H0, eta_max, eta_init_max, t, cdklm_dt) 
    
    cdklm_h0 = None
    cdklm_eta1, u1, v1 = None, None, None
    #if 'cdklm_sim' in globals():
    #    cdklm_sim.cleanUp()